In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
import nltk
from bs4 import BeautifulSoup

In [3]:
db = pd.read_csv('/Users/maurelco/Developer/Python/Projet 4/QueryResults-3.csv')

In [4]:
db

,Id,Title,Body,Tags,Score,ViewCount,FavoriteCount,AnswerCount
0,4019784,Giving a unix process exclusive RW access to a...,<p>Is there a way to sandbox a linux process i...,<linux><ubuntu><process><sandbox><selinux>,2,406,0.0,4
1,4219547,automatic repaint when minimizing window,"<p>I have a JFrame, with two panels, in one pa...",<java><graphics><jframe><jpanel><paint>,1,4225,NaN,3
2,4529530,Is man-in-the-middle attack a security threat ...,"<p>I am no expert in network security, so pard...",<security><ssh><ssh-keys><openssh><man-in-the-...,11,10589,0.0,2
3,4529565,Managing data access in a simple WinForms app,<p>I have a simple WinForms data entry app tha...,<c#><winforms><sqlite><datatable><sqlconnection>,0,1670,NaN,1
4,4529586,Render basic HTML view?,<p>I have a basic Node.js app that I am trying...,<javascript><html><node.js><mongodb><express>,316,516830,0.0,31
...,...,...,...,...,...,...,...,...
49995,68396411,Bypass Vertica ERROR 3326: Execution time exce...,<p>Using the <code>SSIS</code> tool and <code>...,<sql-server><ssas><oledb><sql-server-data-tool...,0,734,NaN,1
49996,68396463,A conflicting conditional operation is current...,"<p>Using s3fs, I am uploading a file to the al...",<python><amazon-web-services><amazon-s3><boto3...,0,670,NaN,2
49997,68396513,Problem in lr_find() in Pytorch fastai course,<p>While following the Jupyter notebooks for t...,<python><machine-learning><deep-learning><pyto...,4,1260,NaN,1
49998,68396523,JSONPatch escape slash '/' from JSONPatch+JSON,<p>I've below JSON and I wanted to update few ...,<java><json><rest><json-patch><http-patch>,1,509,0.0,1


In [5]:
soup = BeautifulSoup(db['Body'], 'html.parser')
soup.p.string

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
for line in db['Body']:
    #TODO: VREER UN BEAUTIFUL SOUP objet
#TODO: recuperer le texte de la balise p
#TODO: remettre dans la ligne de la colonne 'BODY'
soup = BeautifulSoup(db['Body'], 'html.parser')
soup.p.string